### Transform population_by_age.tsv file

##### Reading population_by_age.tsv

In [0]:
population_df = spark.read \
    .csv('/mnt/covid19reportingadfadls/raw/population/population_by_age.tsv', sep=r'\t', header=True)
    
# Here sep=r'\t', it represents a tab character as the separator. The r before the string in Python denotes a raw string. In raw strings, backslashes (\) are treated as literal characters and not as escape characters. So, r'\t' is equivalent to '\\t' in Python.

In [0]:
# display(population_df.limit(5))

"indic_de,geo\time",2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
"PC_Y0_14,AD",14.6,14.5,14.5,15.5,15.5,15.5,:,:,:,:,:,13.9
"PC_Y0_14,AL",24.1,23.3,22.5,21.6,20.7,20.1,19.6,19.0,18.5,18.2,17.7,17.2
"PC_Y0_14,AM",19.0,18.6,18.3,:,:,:,:,19.4,19.6,20.0,20.2,20.2
"PC_Y0_14,AT",15.4,15.1,14.9,14.7,14.6,14.4,14.3,14.3,14.3,14.4,14.4,14.4
"PC_Y0_14,AZ",23.2,22.6,22.6,22.3,22.2,22.3,22.4,22.4,22.5,22.6,22.6,22.4


##### Transforming population file

In [0]:
from pyspark.sql.functions import split, col, regexp_replace, cast
from pyspark.sql.types import DecimalType

In [0]:
population_trans_df = population_df \
    .withColumn('age_group', regexp_replace(split(r'indic_de,geo\time', ',').getItem(0), 'PC_', '')) \
    .withColumn('country_code', split(r'indic_de,geo\time', ',')[1]) \
    .filter('length(country_code) = 2') \
    .select(col('age_group'), col('country_code'), regexp_replace(col('2019 '), '[a-z]', '').alias('percentage_2019').cast(DecimalType(4,2))) \
    .filter(col('percentage_2019').isNotNull())
    
# Here I am selecting 2019 like this, because PySpark interprets 2019 as a number, not a string.

In [0]:
# display(population_trans_df.limit(5))

age_group,country_code,percentage_2019
Y0_14,AD,13.90
Y0_14,AL,17.20
Y0_14,AM,20.20
Y0_14,AT,14.40
Y0_14,AZ,22.40


In [0]:
population_pivot_df = population_trans_df.groupBy('country_code').pivot('age_group').sum('percentage_2019')

In [0]:
display(population_pivot_df.limit(5))

country_code,Y0_14,Y15_24,Y25_49,Y50_64,Y65_79,Y80_MAX
LT,15.10,10.50,32.70,21.90,14.00,5.80
AZ,22.40,14.10,39.10,17.60,5.30,1.50
FI,16.00,11.20,31.20,19.90,16.30,5.50
UA,15.40,9.60,37.60,20.60,12.50,4.30
RO,15.70,10.60,35.60,19.70,13.90,4.70


##### Reading lookup file

In [0]:
lookup_df = spark.read.option('inferSchema', True) \
    .option('header', True) \
    .csv('/mnt/covid19reportingadfadls/lookup/country_lookup/country_lookup.csv')

In [0]:
display(lookup_df.limit(5))

country,country_code_2_digit,country_code_3_digit,continent,population
Aruba,AW,ABW,America,106766
Afghanistan,AF,AFG,Asia,38928341
Angola,AO,AGO,Africa,32866268
Anguilla,AI,AIA,America,15002
Albania,AL,ALB,Europe,2862427


##### Joining population file to lookup file

In [0]:
final_df = population_pivot_df.join(lookup_df, population_pivot_df.country_code == lookup_df.country_code_2_digit, 'left') \
    .select('country', 'country_code_2_digit', 'country_code_3_digit', 'population', col('Y0_14').alias('age_group_0_14'), col('Y15_24').alias('age_group_15_24'), col('Y25_49').alias('age_group_25_49'), col('Y50_64').alias('age_group_50_64'), col('Y65_79').alias('age_group_65_70'), col('Y80_MAX').alias('age_group_80_max'))

##### Write output data to processed container

In [0]:
final_df.write.mode('overwrite').format('parquet').saveAsTable('covid19_processed.population')

In [0]:
%sql
SELECT * FROM covid19_processed.population
LIMIT 5;

country,country_code_2_digit,country_code_3_digit,population,age_group_0_14,age_group_15_24,age_group_25_49,age_group_50_64,age_group_65_70,age_group_80_max
Lithuania,LT,LTU,2794184,15.10,10.50,32.70,21.90,14.00,5.80
Azerbaijan,AZ,AZE,10139175,22.40,14.10,39.10,17.60,5.30,1.50
Finland,FI,FIN,5517919,16.00,11.20,31.20,19.90,16.30,5.50
Ukraine,UA,UKR,43733759,15.40,9.60,37.60,20.60,12.50,4.30
Romania,RO,ROU,19414458,15.70,10.60,35.60,19.70,13.90,4.70


In [0]:
dbutils.notebook.exit('Success')